# Building conversational applications with the Converse API

In this notebook, you learn how to use the flexible Converse API to integrate external capabilities into conversational applications.

Certain conversational applications demand an adaptable sequence of calls to language models and various utilities depending on user input. The Converse API enables building such flexible dialogue agents.

## Environment setup

In this task, you set up your environment.

In [ ]:
#create a service client by name using the default session.
import json
import os
import boto3

bedrock_client = boto3.client('bedrock-runtime')
model_id = "amazon.nova-lite-v1:0"

## MCPClient definition

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

class MCPClient:
    def __init__(self):
        self.session = None
        self.client_context = None
        self.session_context = None
    
    async def connect(self):
        server_params = StdioServerParameters(
            command="uv", 
            args=["run", "module9-mcp_server.py"]
        )
        
        # Keep reference to context managers
        self.client_context = stdio_client(server_params)
        read, write = await self.client_context.__aenter__()
        
        self.session_context = ClientSession(read, write)
        self.session = await self.session_context.__aenter__()
        await self.session.initialize()
        
        print("Connected to MCP server!")
    
    async def disconnect(self):
        if self.session_context:
            await self.session_context.__aexit__(None, None, None)
        if self.client_context:
            await self.client_context.__aexit__(None, None, None)
        print("Disconnected from MCP server")

In [ ]:
# Create global client instance
mcp_client = MCPClient()

# Connect to the server
await mcp_client.connect()

# Disconnect from the server
# await mcp_client.disconnect()

## Get the tool list from the MCP server

In [ ]:
async def get_tool_list():
    mcp_list_tools_response = await mcp_client.session.list_tools()
    tools = []
    for tool in mcp_list_tools_response.tools:
        tools.append({
            "toolSpec": {
                "name": tool.name,
                "description": tool.description,
                "inputSchema": {"json": tool.inputSchema}
            }
        })
    return {
        "tools": tools
    }

Next, you create a function to call the converse API to converse with an Amazon Bedrock model to generate text responses.

In [ ]:
async def generate_text(bedrock_client, model_id, message_list):
    
    response = bedrock_client.converse(
        modelId=model_id,
        messages=message_list,
        toolConfig=await get_tool_list(),
        system=[{"text": "You are a helpful assistant. You can use tools to help you answer questions. You should always use tools to help you with calculations. You can also use tools to help you search the internet and find current information."}]
    )
    
    return response


Next, you create a function to run the tools that the text generated by the Converse API described implementing the ACT of ReACT. This function is the core logic for the interpretation of the text generation response:
* It loops across all content blocks of the response
* It calls the functions of the tools defined above based on what the model told it to do.
* It appropriately adds the output of the function calls of the tools defined above.

In [ ]:
async def run_the_tools(response_message):
    
    response_content_blocks = response_message['content']
    
    follow_up_content_blocks = []
    
    for content_block in response_content_blocks:
        if 'toolUse' in content_block:
            tool = content_block['toolUse']
            
            # Execute tool call
            result = await mcp_client.session.call_tool(tool['name'], tool['input'])
            follow_up_content_blocks.append({
                "toolResult": {
                    "toolUseId": tool['toolUseId'],
                    "content": [{"json": result.structuredContent}]
                }
            })
     
    if len(follow_up_content_blocks) > 0:
        follow_up_message = {
            "role": "user",
            "content": follow_up_content_blocks,
        }
        
        return follow_up_message
    else:
        return None


The `converse` function run an infinite loop and implements the following logic:
1. Calls the `generate_text` function for the Converse API of Bedrock.
2. Appends the output of the model in the message list as the Converse API will need the context.
2. Calls the `run_the_tools` function to run the tools based on the response of #1.
3. Appends the output of the `run_the_tools` to the message list as the Converse API will need the answer in the context.
4. Decides if it needs to go back to #1 if there was a tool used or end the loop

It finally sends back the list of messages containing the whole conversation.

In [ ]:
async def converse(prompt, bedrock_client, model_id):

    message_list = [
        {
            "role": "user",
            "content": [ { "text": prompt } ]
        }
    ]
    
    while True:
        response = await generate_text(bedrock_client, model_id, message_list)
        
        response_message = response['output']['message']
        message_list.append(response_message)
        
        follow_up_message = await run_the_tools(response_message)
        
        if follow_up_message is None:
            # No remaining work to do, return final response to user
            break
        else:
            message_list.append(follow_up_message)
            
    return message_list

Next, you send your query to the `converse` function that orchestrates the end-to-end flow to have a conversation with the Amazon Bedrock model leveraging the integrated tools.

In [ ]:
query = "What is Amazon SageMaker? What is launch year multiplied by 2"

messages = await converse(query, bedrock_client, model_id)

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=4))

You have successfully integrated Amazon Bedrock with custom capabilities by utilizing Converse API, the tool framework and MCP.